<a href="https://colab.research.google.com/github/diegoalvarezeo/Fakenewss/blob/main/espaindata1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
#mostrando palabras vacías
print(stopwords.words('spanish'))

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [4]:
#Cargando nuestro dataset
news_dataset = pd.read_csv('espanoln.csv')

In [5]:
#Cantidad de noticias
news_dataset.shape

(10000, 6)

In [6]:
#Estructura primeras 5 noticias
news_dataset.head()

,Unnamed: 0,authors,title,summary,text,label
0,0,Christian Marquardt,La confianza empresarial de Alemania vuelve a ...,CHRISTIAN MARQUARDT / POOL EFELa confianza emp...,Angela Merkel ayer en Berlin tras la cumbre te...,1
1,1,NaN,El Covid dejará 10 cambios que te afectarán,NaN,\n\n\n\n\nEnrique Quemada Clariana\nFusiones y...,0
2,2,"Ediciones Cinco Días, Getty Images",Quiero ser mi propio jefe,¿Cuántas veces no ha querido ser su propio jef...,¿Cuántas veces no ha querido ser su propio jef...,1
3,3,"Pedro Gorospe, Javier Etxezarreta",Illa estima que entre mayo y junio pueden esta...,Javier Etxezarreta / EFEEl ministro de Sanidad...,"El ministro de Sanidad, Salvador Illa, momento...",0
4,4,Web Financial Group,El Congreso aprueba los Presupuestos con 188 v...,El Pleno del Congreso ha aprobado el proyecto ...,El Pleno del Congreso ha aprobado el proyecto ...,1


In [7]:
#Verificando cantidad de datos vacios
news_dataset.isnull().sum()

Unnamed: 0       0
authors       1419
title            0
summary         44
text             0
label            0
dtype: int64

In [8]:
#transformando datos vacios en cadenas vacias
news_dataset = news_dataset.fillna('')

In [9]:
#Separando los datos, concatenación de fila autor y titulo
news_dataset['content'] = news_dataset['authors']+' '+news_dataset['title']

In [10]:
print(news_dataset['content'])

0       Christian Marquardt La confianza empresarial d...
1             El Covid dejará 10 cambios que te afectarán
2       Ediciones Cinco Días, Getty Images Quiero ser ...
3       Pedro Gorospe, Javier Etxezarreta Illa estima ...
4       Web Financial Group El Congreso aprueba los Pr...
                              ...                        
9995    María Antonia Sánchez-Vallejo, John Minchillo ...
9996    Web Financial Group, Sergio García La FDA da l...
9997    Pablo G. Bejerano, R. J. C. Inrupt: el plan B ...
9998    Redacción, Estrategias De Inversión Las bolsas...
9999    Raquel Pascual Cortés CSIF pide contratar 106....
Name: content, Length: 10000, dtype: object


In [11]:
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [12]:
print(X)
print(Y)

      Unnamed: 0  ...                                            content
0              0  ...  Christian Marquardt La confianza empresarial d...
1              1  ...        El Covid dejará 10 cambios que te afectarán
2              2  ...  Ediciones Cinco Días, Getty Images Quiero ser ...
3              3  ...  Pedro Gorospe, Javier Etxezarreta Illa estima ...
4              4  ...  Web Financial Group El Congreso aprueba los Pr...
...          ...  ...                                                ...
9995        9995  ...  María Antonia Sánchez-Vallejo, John Minchillo ...
9996        9996  ...  Web Financial Group, Sergio García La FDA da l...
9997        9997  ...  Pablo G. Bejerano, R. J. C. Inrupt: el plan B ...
9998        9998  ...  Redacción, Estrategias De Inversión Las bolsas...
9999        9999  ...  Raquel Pascual Cortés CSIF pide contratar 106....

[10000 rows x 6 columns]
0       1
1       0
2       1
3       0
4       1
       ..
9995    0
9996    1
9997    0
9998    

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('spanish')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [15]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [16]:
#Datos completamente limpios
print(news_dataset['content'])

0       christian marquardt confianza empresari aleman...
1                            covid dejar cambio afectar n
2       edicion cinco d as getti imag quiero ser propi...
3       pedro gorosp javier etxezarreta illa estima ma...
4       web financi group congreso aprueba presupuesto...
                              ...                        
9995    mar antonia s nchez vallejo john minchillo est...
9996    web financi group sergio garc fda da luz verd ...
9997    pablo g bejerano r j c inrupt plan b internet ...
9998    redacci n estrategia inversi n bolsa europea r...
9999    raquel pascual cort s csif pide contratar empl...
Name: content, Length: 10000, dtype: object


In [17]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [18]:
print(X)

['christian marquardt confianza empresari alemania vuelv caer noviembr tra interrumpirs recuperaci n econ mica'
 'covid dejar cambio afectar n'
 'edicion cinco d as getti imag quiero ser propio jefe' ...
 'pablo g bejerano r j c inrupt plan b internet berner lee echa andar'
 'redacci n estrategia inversi n bolsa europea rebotan ibex intenta recuperar punto'
 'raquel pascual cort s csif pide contratar empleado p blico legislatura sanidad d cada']


In [19]:
print(Y)

[1 0 1 ... 0 1 0]


In [20]:
Y.shape

(10000,)

In [21]:
#Convertiendo los datos en forma de texto a numerico(Entendible por el modelo)
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [22]:
#Datos convertidos
print(X)

  (0, 14357)	0.24061512442433144
  (0, 13594)	0.17742911070791573
  (0, 11443)	0.2182262090163248
  (0, 9564)	0.24061512442433144
  (0, 8836)	0.23686549860017486
  (0, 8554)	0.37442263125641434
  (0, 7319)	0.37442263125641434
  (0, 4804)	0.2819164854843236
  (0, 4592)	0.2328997112750427
  (0, 3119)	0.2858538982721981
  (0, 2681)	0.33837456799890714
  (0, 2158)	0.2819164854843236
  (0, 553)	0.24332939327936148
  (1, 3834)	0.5634997303601084
  (1, 3450)	0.33903979036394155
  (1, 2220)	0.4814931433962023
  (1, 344)	0.5793827985852736
  (2, 12528)	0.28164677237872404
  (2, 11150)	0.48917859782600703
  (2, 10955)	0.4238382466975975
  (2, 7512)	0.43838276839144497
  (2, 6927)	0.2860840493095398
  (2, 6199)	0.2860840493095398
  (2, 4609)	0.27633330974400333
  (2, 2734)	0.18634236789104328
  :	:
  (9997, 4572)	0.3324022462789719
  (9997, 1731)	0.4049345758039711
  (9997, 1672)	0.3659489323487363
  (9997, 792)	0.3659489323487363
  (9998, 11458)	0.25230470126337473
  (9998, 11447)	0.365679672225

In [23]:
#Dividiendo los datos en datos de entrenamiento y datos de testeo
#Datos de entrenamiento 80%
#Datos de testeo 20%
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [24]:
model = LogisticRegression()

In [25]:
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
# Precision del modelo en datos de entrenamiento
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [27]:
print('Precisión datos de entrenamiento : ', training_data_accuracy)

Precisión datos de entrenamiento :  0.834375


In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
confusion_matrix(X_train_prediction, Y_train)

array([[3344,  669],
       [ 656, 3331]])

In [30]:
# Precision de los datos de testeo
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [31]:
print('Precisión de los datos de testeo : ', test_data_accuracy)

Precisión de los datos de testeo :  0.5045


In [32]:
print(Y_test[0])
print(Y_test[5])
print(Y_test[10])

0
1
1


Segundo Modelo de Entrenamiento
**Clasificador de árbol de decisión**




.

In [33]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, Y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [35]:
# Precision del modelo en datos de entrenamiento
X_train_predictionn = dtc.predict(X_train)
training_data_accuracyy = accuracy_score(X_train_predictionn, Y_train)

In [36]:
print('Precisión datos de entrenamiento : ', training_data_accuracyy)

Precisión datos de entrenamiento :  0.9755


In [37]:
# Precision de los datos de testeo
X_test_predictionn = dtc.predict(X_test)
test_data_accuracyy = accuracy_score(X_test_predictionn, Y_test)

In [38]:
print('Precisión de los datos de testeo : ', test_data_accuracyy)

Precisión de los datos de testeo :  0.5095


El resultado es practicamente el mismo

TESTEO PRIMER MODELO ELEGIDO REGRESIÓN LOGISTICA




In [ ]:
print(Y_test[5])
# 1= Falso

1


In [ ]:
#Codigo Verificador del modelo, entrega el resultado si la noticia es Falsa(1) o verdadera(0)
X_new = X_test[5]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('La noticia es real')
else:
  print('La noticia es falsa')

[1]
La noticia es falsa
